In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F  # Importar softmax

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Modelo de Flor con regresión logística

In [ ]:
# Ruta al modelo guardado
ruta_local = '/content/drive/My Drive/modelo_guardado_entrenamiento32_Flor'

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(ruta_local)

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(ruta_local)

print("Modelo y tokenizer cargados correctamente.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/modelo_guardado_entrenamiento32_Flor and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo y tokenizer cargados correctamente.


In [ ]:
label_map = {0: 'tini', 1: 'spinetta', 2: 'callejero', 3: 'dillom'}  # Ajusta según sea necesario

In [ ]:
# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase

El modelo tiene 4 clases.


In [ ]:
'''
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")
'''

'\nwhile True:\n    print("\nIntroduce la letra de una canción (o escribe \'salir\' para terminar):")\n    cancion_input = input("> ")\n    if cancion_input.lower() == \'salir\':\n        print("¡Hasta luego!")\n        break\n\n    # Llamar a la función para predecir el artista\n    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)\n    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")\n'

#Modelos de Vic
##con 18 canciones y learning rate dinámico

In [ ]:
# Ruta al modelo guardado
ruta_local = '/content/drive/My Drive/modelo_guardado'

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(ruta_local)

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(ruta_local)

print("Modelo y tokenizer cargados correctamente.")

# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase



Modelo y tokenizer cargados correctamente.
El modelo tiene 4 clases.


In [ ]:
'''
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")
'''

'\nwhile True:\n    print("\nIntroduce la letra de una canción (o escribe \'salir\' para terminar):")\n    cancion_input = input("> ")\n    if cancion_input.lower() == \'salir\':\n        print("¡Hasta luego!")\n        break\n\n    # Llamar a la función para predecir el artista\n    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)\n    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")\n'

## 31 canciones por artista learning rate dinamico

In [ ]:
# Ruta al modelo guardado
ruta_local = '/content/drive/My Drive/modelo_guardado_entrenamiento31_dinamico'

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(ruta_local)

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(ruta_local)

print("Modelo y tokenizer cargados correctamente.")

# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase



Modelo y tokenizer cargados correctamente.
El modelo tiene 4 clases.


In [ ]:
'''
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")
'''

'\nwhile True:\n    print("\nIntroduce la letra de una canción (o escribe \'salir\' para terminar):")\n    cancion_input = input("> ")\n    if cancion_input.lower() == \'salir\':\n        print("¡Hasta luego!")\n        break\n\n    # Llamar a la función para predecir el artista\n    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)\n    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")\n'

## 31 canciones por artista learning rate estatico

In [ ]:
# Ruta al modelo guardado
ruta_local = '/content/drive/My Drive/modelo_guardado_entrenamiento31'

# Cargar el modelo
model = AutoModelForSequenceClassification.from_pretrained(ruta_local)

# Cargar el tokenizer
tokenizer = AutoTokenizer.from_pretrained(ruta_local)

print("Modelo y tokenizer cargados correctamente.")

# Número de clases en el modelo
num_clases = model.config.num_labels
print(f"El modelo tiene {num_clases} clases.")

def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase



Modelo y tokenizer cargados correctamente.
El modelo tiene 4 clases.


In [ ]:
'''
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    # Llamar a la función para predecir el artista
    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)
    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")
'''

'\nwhile True:\n    print("\nIntroduce la letra de una canción (o escribe \'salir\' para terminar):")\n    cancion_input = input("> ")\n    if cancion_input.lower() == \'salir\':\n        print("¡Hasta luego!")\n        break\n\n    # Llamar a la función para predecir el artista\n    artista_predicho = predecir_artista(cancion_input, tokenizer, model, label_map)\n    print(f"\nEl modelo predice que esta canción es de: {artista_predicho}\n")\n'

#Probemos los 4 al mismo tiempo!

In [ ]:
# Rutas a los modelos guardados
rutas_modelos = {
    "modelo Flor regresion logistica": "/content/drive/My Drive/modelo_guardado_entrenamiento32_Flor",
    "modelo Vic con 18 canciones": "/content/drive/My Drive/modelo_guardado",
    "modelo Vic con 31 canciones LR estatico": "/content/drive/My Drive/modelo_guardado_entrenamiento31",
    "modelo Vic con 31 canciones LR dinamico": "/content/drive/My Drive/modelo_guardado_entrenamiento31_dinamico",
}

# Diccionarios para almacenar modelos y tokenizers
modelos = {}
tokenizers = {}

# Cargar los modelos y tokenizers
for nombre, ruta in rutas_modelos.items():
    modelos[nombre] = AutoModelForSequenceClassification.from_pretrained(ruta)
    tokenizers[nombre] = AutoTokenizer.from_pretrained(ruta)
    print(f"Modelo y tokenizer para {nombre} cargados correctamente.")

# Número de clases por modelo (se asume que todos tienen el mismo número de clases)
num_clases = modelos["modelo Flor regresion logistica"].config.num_labels
print(f"Cada modelo tiene {num_clases} clases.")

# Función para predecir artista con un modelo específico
def predecir_artista(cancion, tokenizer, model, label_map):
    # Detectar el dispositivo en el que se encuentra el modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Asegurarse de que el modelo esté en el dispositivo

    # Tokenizar la entrada y moverla al mismo dispositivo
    inputs = tokenizer(cancion, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Mover entradas al dispositivo

    # Pasar la entrada por el modelo y obtener las predicciones
    model.eval()  # Configurar el modelo en modo de evaluación
    with torch.no_grad():  # No calcular gradientes para una inferencia más rápida
        outputs = model(**inputs)
        logits = outputs.logits

    # Aplicar softmax a los logits para obtener probabilidades
    probs = F.softmax(logits, dim=-1)

    # Obtener la predicción de la clase (índice con la mayor probabilidad)
    pred_idx = probs.argmax(dim=-1).item()

    # Obtener las probabilidades de cada clase
    probabilidades = probs.squeeze().cpu().numpy()

    # Crear un diccionario de probabilidades con las clases
    probabilidades_clase = {label_map[i]: probabilidad for i, probabilidad in enumerate(probabilidades)}

    # Validar si el índice está en el rango de las clases posibles
    if pred_idx in label_map:
        artista_predicho = label_map[pred_idx]
    else:
        artista_predicho = "Desconocido"  # Si el índice no es válido

    return artista_predicho, probabilidades_clase


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/modelo_guardado_entrenamiento32_Flor and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modelo y tokenizer para modelo Flor regresion logistica cargados correctamente.
Modelo y tokenizer para modelo Vic con 18 canciones cargados correctamente.
Modelo y tokenizer para modelo Vic con 31 canciones LR estatico cargados correctamente.
Modelo y tokenizer para modelo Vic con 31 canciones LR dinamico cargados correctamente.
Cada modelo tiene 4 clases.


In [ ]:
while True:
    print("\nIntroduce la letra de una canción (o escribe 'salir' para terminar):")
    cancion_input = input("> ")
    if cancion_input.lower() == 'salir':
        print("¡Hasta luego!")
        break

    print("\nRealizando predicciones con los 4 modelos...")

    # Mapa de etiquetas para los artistas
    label_map = {0: "Tini", 1: "Spinetta", 2: "Callejero", 3: "Dillom"}

    resultados = {}  # Diccionario para almacenar resultados

    # Realizar predicción con cada modelo
    for nombre_modelo in modelos.keys():
        tokenizer = tokenizers[nombre_modelo]
        model = modelos[nombre_modelo]

        # Llamar a la función de predicción
        artista_predicho, probabilidades_clase = predecir_artista(cancion_input, tokenizer, model, label_map)

        # Guardar resultados
        resultados[nombre_modelo] = {
            "artista_predicho": artista_predicho,
            "probabilidades": probabilidades_clase
        }

    # Mostrar resultados para cada modelo
    for nombre_modelo, resultado in resultados.items():
        print(f"\nResultados del {nombre_modelo}:")
        print(f"Artista predicho: {resultado['artista_predicho']}")
        print("Probabilidades por artista:")
        for artista, probabilidad in resultado['probabilidades'].items():
            print(f"{artista}: {probabilidad:.2f}")




Introduce la letra de una canción (o escribe 'salir' para terminar):

Realizando predicciones con los 4 modelos...

Resultados del modelo Flor regresion logistica:
Artista predicho: Callejero
Probabilidades por artista:
Tini: 0.16
Spinetta: 0.21
Callejero: 0.32
Dillom: 0.31

Resultados del modelo Vic con 18 canciones:
Artista predicho: Callejero
Probabilidades por artista:
Tini: 0.27
Spinetta: 0.08
Callejero: 0.35
Dillom: 0.30

Resultados del modelo Vic con 31 canciones LR estatico:
Artista predicho: Tini
Probabilidades por artista:
Tini: 0.90
Spinetta: 0.02
Callejero: 0.02
Dillom: 0.06

Resultados del modelo Vic con 31 canciones LR dinamico:
Artista predicho: Tini
Probabilidades por artista:
Tini: 0.44
Spinetta: 0.05
Callejero: 0.10
Dillom: 0.40

Introduce la letra de una canción (o escribe 'salir' para terminar):

Realizando predicciones con los 4 modelos...

Resultados del modelo Flor regresion logistica:
Artista predicho: Callejero
Probabilidades por artista:
Tini: 0.16
Spinetta: 